## Advanced DataFrames Practice

In [6]:
# imports
import numpy as np
import pandas as pd
from env import host, user, password, get_db_url

np.random.seed(123)

In [7]:
# Create list of values for names column.

students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))


In [8]:
# Construct the DataFrame using the above lists and arrays.

df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})


In [9]:
url = get_db_url('employees')

In [11]:
# query employees table from employees database
emp_query = '''

SELECT * 
FROM employees e;'''


In [12]:
employees = pd.read_sql(emp_query, url)
employees.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [13]:
tit_query = '''

SELECT * 
FROM titles'''

In [14]:
titles = pd.read_sql(tit_query, url)

In [15]:
titles.head()

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01


In [17]:
employees.shape

(300024, 6)

In [18]:
titles.shape

(443308, 4)

In [19]:
# there are a lot more titles than employees. This makes sense if many
# employees have changed titles during their tenure

In [23]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300024 entries, 0 to 300023
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   emp_no      300024 non-null  int64 
 1   birth_date  300024 non-null  object
 2   first_name  300024 non-null  object
 3   last_name   300024 non-null  object
 4   gender      300024 non-null  object
 5   hire_date   300024 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.7+ MB


In [24]:
titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443308 entries, 0 to 443307
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   emp_no     443308 non-null  int64 
 1   title      443308 non-null  object
 2   from_date  443308 non-null  object
 3   to_date    443308 non-null  object
dtypes: int64(1), object(3)
memory usage: 13.5+ MB


In [26]:
# how many unique titles are there? 7
titles.title.nunique()

7

In [29]:
# the oldest date in the to_date column is 3/1/1985
# it appears that the protocol is to use 1/1/9999 as a proxy for 'presently' or
# some future time
titles.to_date.min(), titles.to_date.max()

(datetime.date(1985, 3, 1), datetime.date(9999, 1, 1))

#### Indexing and Subsetting
- Like the pandas Series object, the pandas DataFrame object supports both position- and label-based indexing using the indexing operator [].
- I will demonstrate concrete examples of indexing using the indexing operator [] alone and with the .loc and .iloc attributes below.


In [30]:
# Choose only two columns for my subset.

df[['name', 'classroom']]



,name,classroom
0,Sally,A
1,Jane,B
2,Suzie,A
3,Billy,B
4,Ada,A
5,John,B
6,Thomas,A
7,Marie,A
8,Albert,A
9,Richard,A


In [31]:
# can pass a boolean Series to the indexing operator as a selector
bools = df.name.str.startswith('A')
bools

0     False
1     False
2     False
3     False
4      True
5     False
6     False
7     False
8      True
9     False
10    False
11     True
Name: name, dtype: bool

In [32]:
df[bools]

,name,math,english,reading,classroom
4,Ada,77,92,98,A
8,Albert,92,62,87,A
11,Alan,92,62,72,A


- We can use the .loc attribute to select specific rows AND columns by index label. The index label can be a number, but it can also be a string label. This method offers a lot of flexibility! The .loc attribute's indexing is inclusive and uses an index label, not position.

 - this looks like `df.loc[row_indexer, column_indexer]` in general form

In [34]:
# select all the rows and a subset of the columns. Note .loc is inclusive.
df.loc[:, 'math':'reading']

,math,english,reading
0,62,85,80
1,88,79,67
2,94,74,95
3,98,96,88
4,77,92,98
5,79,76,93
6,82,64,81
7,93,63,90
8,92,62,87
9,69,80,94


In [35]:
# I can use a boolean Series as a selector with .loc, too, but I can choose rows and columns.

df.loc[bools, 'name': 'reading']


,name,math,english,reading
4,Ada,77,92,98
8,Albert,92,62,87
11,Alan,92,62,72


- We can use the `.iloc` attribute to select specific rows and colums by index position. .iloc does not accept a boolean Series as a selector like `.loc` does. It takes in integers representing index position and is NOT inclusive.
-  basic syntax: `df.iloc[row_indexer, column_indexer]`




In [36]:
# Notice the exclusive behavior of the indexing.

df.iloc[:3]


,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A


In [38]:
# rows 0, 1, 2 and columns 1 and 2 (excluding 0, 3, and 4)
df.iloc[:3, 1:3]


,math,english
0,62,85
1,88,79
2,94,74


#### Aggregating
- The `.agg` method lets us specify a way to aggregate a series of numerical values. We pass an aggregate function or list of functions to the method that we want applied to a Series.



In [40]:
# can pass lists of columns to the indexer and a list of aggregation functions to .agg
df[['english', 'reading', 'math']].agg(['mean', 'min', 'max'])


,english,reading,math
mean,77.666667,86.5,84.833333
min,62.000000,67.0,62.000000
max,99.000000,98.0,98.000000


#### .groupby

The `.groupby()` method is used to create a grouped object, which we can then apply an aggregation on. For example, if we wanted to know the highest math grade from each classroom:

In [42]:
df.groupby('classroom').math.max()

classroom
A    94
B    98
Name: math, dtype: int64

- We can group by multiple columns as well. To demonstrate, we'll create a boolean column named passing_math, then group by the combination of our new feature, passing_math, and the classroom and calculate the average reading grade and the number of individuals in each subgroup.



`np.where()` we can create a column based on a condition using np.where()
- general syntax: `np.where(condition, this_where_True, this_where_False)`


In [43]:
df['passing_math'] = np.where(df.math < 70, 'failing', 'passing')

In [44]:
df.head()

,name,math,english,reading,classroom,passing_math
0,Sally,62,85,80,A,failing
1,Jane,88,79,67,B,passing
2,Suzie,94,74,95,A,passing
3,Billy,98,96,88,B,passing
4,Ada,77,92,98,A,passing


In [50]:
grade_groups = df.drop(columns='name').groupby(['passing_math', 'classroom']).reading.agg(['mean', 'count'])
grade_groups

mean  count
passing_math classroom                  
failing      A          87.000000      2
passing      A          87.166667      6
             B          85.250000      4

In [51]:
# I can even clean up my columns to make my calculations clearer.

grade_groups.columns = ['avg_reading_grade', 'count_of_students']
grade_groups


avg_reading_grade  count_of_students
passing_math classroom                                      
failing      A                  87.000000                  2
passing      A                  87.166667                  6
             B                  85.250000                  4